In [27]:
import pandas as pd
import numpy as np

In [28]:
df = pd.read_csv("intents_data.csv")
df.head(5)

Text      Intent
0  hello  small_talk
1     hi  small_talk
2    hey  small_talk
3     yo  small_talk
4   heya  small_talk

In [29]:
df.Intent.value_counts()

Intent
small_talk    1000
quit          1000
thanks        1000
help          1000
predict       1000
get           1000
set           1000
calculate     1000
definition    1000
Name: count, dtype: int64

In [30]:
df.isna().sum()

Text      0
Intent    0
dtype: int64

In [31]:
intent = df.Intent.map({
    "small_talk": 0,
    "thanks": 1, 
    "help": 2,     
    "predict": 3,
    "get": 4,
    "set": 5,        
    "calculate": 6,
    "definition": 7,
    "quit": 8
})

intent

0       0
1       0
2       0
3       0
4       0
       ..
8995    7
8996    7
8997    7
8998    7
8999    7
Name: Intent, Length: 9000, dtype: int64

In [32]:
import spacy

nlp = spacy.load("en_core_web_sm")

In [33]:
def preprocess(text):
    doc = nlp(text)

    filter_word = []

    for token in doc:
        # if token.is_stop or token.is_punct:
        #     continue
        filter_word.append(token.lemma_)

    return " ".join(filter_word)

In [34]:
text = df.Text.apply(preprocess)

In [35]:
text

0                                          hello
1                                             hi
2                                            hey
3                                             yo
4                                           heya
                          ...                   
8995           what be another word for ' word '
8996      be there a technical term for ' word '
8997    let I see ' word ' define in simple term
8998            could you clarify ' word ' for I
8999                       what do ' word ' mean
Name: Text, Length: 9000, dtype: object

In [36]:
df1 = pd.concat([text, intent], axis="columns")

In [37]:
df1

Text  Intent
0                                        hello       0
1                                           hi       0
2                                          hey       0
3                                           yo       0
4                                         heya       0
...                                        ...     ...
8995         what be another word for ' word '       7
8996    be there a technical term for ' word '       7
8997  let I see ' word ' define in simple term       7
8998          could you clarify ' word ' for I       7
8999                     what do ' word ' mean       7

[9000 rows x 2 columns]

In [38]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [39]:
bag_of_word = CountVectorizer()
bag_of_ngram = CountVectorizer(ngram_range=(1, 2))
tf_idf = TfidfVectorizer()

x_bag_of_word = bag_of_word.fit_transform(df1.Text.values).toarray()
x_bag_of_ngram = bag_of_ngram.fit_transform(df1.Text.values).toarray()
x_tf_idf = tf_idf.fit_transform(df1.Text.values).toarray()

In [40]:
y = df1.Intent

In [41]:
from sklearn.model_selection import train_test_split

X_train_bag_of_word, X_test_bag_of_word, y_train_bag_of_word, y_test_bag_of_word = train_test_split(x_bag_of_word, y, test_size=.2, stratify=df1.Intent)
X_train_bag_of_ngram, X_test_bag_of_ngram, y_train_bag_of_ngram, y_test_bag_of_ngram = train_test_split(x_bag_of_ngram, y, test_size=.2, stratify=df1.Intent)
X_train_tf_idf, X_test_tf_idf, y_train_tf_idf, y_test_tf_idf = train_test_split(x_tf_idf, y, test_size=.2, stratify=df1.Intent)

## Machine Learning

In [42]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from  sklearn.tree import DecisionTreeClassifier
from  sklearn.svm import SVC
from  sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

In [43]:
model_params = {
    'svm': {
        'model': SVC(),
        'params' : {
            "C": [1.0, 2.0, 3.0],
            "kernel": ["linear", "poly", "rbf", "sigmoid"],
            "degree": [1, 2, 3],
            "gamma": ["scale", "auto"]
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            "n_estimators": [10, 50, 100, 200],
            "criterion": ["gini", "entropy", "log_loss"],

        }
    },
    'gradient_boosting' : {
        'model': GradientBoostingClassifier(),
        'params': {
            "n_estimators": [100, 50, 10],
            "learning_rate": [1.0, .5, .1, .01],
            "max_depth": [3, 2, 1]
        }
    },
    'decision_tree': {
        'model': DecisionTreeClassifier(),
        'params' : {
            "criterion": ["gini", "entropy", "log_loss"],
            "splitter": ["best", "random"]
        }  
    },
    'naive_bayes': {
        'model': MultinomialNB(),
        'params' : {
            
        }
    },
    'knn' : {
        'model': KNeighborsClassifier(),
        'params': {
            "n_neighbors": [5, 10, 50],
            "weights": ["uniform", "distance"],
            "algorithm": ["auto", "ball_tree", "kd_tree", "brute"]
        }
    }
}

In [44]:
from sklearn.model_selection import GridSearchCV

In [56]:
# scores = []

# for model_name, mp in model_params.items():
#     clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
#     clf.fit(X_train_bag_of_word, y_train_bag_of_word)
#     scores.append({
#         'model': model_name,
#         'best_score': clf.best_score_,
#         'best_params': clf.best_params_
#     })


# df = pd.DataFrame(scores)
# print(df)

In [46]:
from sklearn.metrics import classification_report

In [57]:
for model_name, mp in model_params.items():
    mp["model"].fit(X_train_bag_of_word, y_train_bag_of_word)

KeyboardInterrupt: 

In [49]:
print("Bag of word")
for model_name, mp in model_params.items():
    y_pred = mp["model"].predict(X_test_bag_of_word)
    print(model_name)
    print(classification_report(y_test_bag_of_word, y_pred))
    print("---------------")

In [ ]:
# scores = []

# for model_name, mp in model_params.items():
#     clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
#     clf.fit(X_train_bag_of_ngram, y_train_bag_of_ngram)
#     scores.append({
#         'model': model_name,
#         'best_score': clf.best_score_,
#         'best_params': clf.best_params_
#     })

# df = pd.DataFrame(scores)
# print(df)

               model  best_score  \
0                svm    0.995694   
1      random_forest    0.996250   
2  gradient_boosting    0.996250   
3      decision_tree    0.995694   
4        naive_bayes    0.989861   
5                knn    0.996944   

                                         best_params  
0  {'C': 1.0, 'degree': 1, 'gamma': 'scale', 'ker...  
1          {'criterion': 'gini', 'n_estimators': 10}  
2  {'learning_rate': 0.5, 'max_depth': 3, 'n_esti...  
3          {'criterion': 'gini', 'splitter': 'best'}  
4                                                 {}  
5  {'algorithm': 'auto', 'n_neighbors': 5, 'weigh...  


In [ ]:
for model_name, mp in model_params.items():
    mp["model"].fit(X_train_bag_of_ngram, y_train_bag_of_ngram)

In [ ]:
print("Bag of word")
for model_name, mp in model_params.items():
    y_pred = mp["model"].predict(X_test_bag_of_ngram)
    print(model_name)
    print(classification_report(y_test_bag_of_ngram, y_pred))
    print("---------------")

In [ ]:
# scores = []

# for model_name, mp in model_params.items():
#     clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
#     clf.fit(X_train_tf_idf, y_train_tf_idf)
#     scores.append({
#         'model': model_name,
#         'best_score': clf.best_score_,
#         'best_params': clf.best_params_
#     })

# df = pd.DataFrame(scores)
# print(df)

               model  best_score  \
0                svm    0.994722   
1      random_forest    0.995556   
2  gradient_boosting    0.995278   
3      decision_tree    0.995556   
4        naive_bayes    0.979167   
5                knn    0.996250   

                                         best_params  
0  {'C': 1.0, 'degree': 3, 'gamma': 'scale', 'ker...  
1          {'criterion': 'gini', 'n_estimators': 50}  
2  {'learning_rate': 0.5, 'max_depth': 3, 'n_esti...  
3          {'criterion': 'gini', 'splitter': 'best'}  
4                                                 {}  
5  {'algorithm': 'ball_tree', 'n_neighbors': 10, ...  


In [ ]:
for model_name, mp in model_params.items():
    mp["model"].fit(X_train_tf_idf, y_train_tf_idf)

In [ ]:
print("TF-IDF")
for model_name, mp in model_params.items():
    y_pred = mp["model"].predict(X_test_tf_idf)
    print(model_name)
    print(classification_report(y_test_tf_idf, y_pred))
    print("---------------")

## Deep Learning